In [ ]:
import time
import numpy as np
import skvideo.io
from matplotlib import pyplot as plt
from skimage.color import rgb2gray
from skimage.measure import label
from skimage.morphology import remove_small_objects, erosion, dilation, disk

# Ignore warnings for simplicity
import warnings
warnings.filterwarnings('ignore')

from trackbox import *
import multiprocessing
num_cores = multiprocessing.cpu_count()
print('num_cores for multiprocessing: ', num_cores)

### 0. Load the video
* Change `filename = "name_of_video.wmv"` to the name of your own video.
* Change `subsample=1` to `n` to speed-up the video by `n` times.
* Change `start` and `end` to the MM:SS format (e.g., `3:11`) to trim the video.

In [ ]:
start = time.perf_counter()
filename = "name_of_video.wmv"
height, width = 28, 65 # height and width of the chamber in cm
p = multiprocessing.Pool(num_cores)
kwargs = {
    "subsample": 2,   # subsampling ratio of the video
    "start": "0:09",  # start time of the trimmed video 
    "end": "4:09",    # end time of the trimmed video 
    "p": p,
    "num_cores": num_cores
}    
video, video_gray, frame_rate = load_video(filename, **kwargs)
%matplotlib inline
show_rand_imgs(video, 3)
num_frames = video_gray.shape[0]
end = time.perf_counter()
print("Data loading time: %.2f s" % (end-start))

### 0.1 Crop the video (*optional*)
If you do **Not** want to crop the video, skip the following two cells

In [ ]:
# If you do **not** want to crop the video, skip this cell.
# If you run this cell, please click the "STOP" size after rectangle selection.
%matplotlib notebook
from matplotlib.widgets import RectangleSelector

global coord
def region_select_callback(eclick, erelease):
    global coord
    'eclick and erelease are the press and release events'
    x1, y1 = eclick.xdata, eclick.ydata
    x2, y2 = erelease.xdata, erelease.ydata
    x1, y1 = int(x1), int(y1)
    x2, y2 = int(x2), int(y2)
    title_string = "Coordinate of the selected rectangle (%d, %d) --> (%d, %d)\n" % (x1, y1, x2, y2)
    plt.title(title_string)
    coord = [x1, x2, y1, y2]
    
%matplotlib notebook   
image_temp = video[0].copy()
fig, current_ax = plt.subplots()    # make a new plotting range
print("\n click  -->  release")
plt.imshow(image_temp)

# drawtype is 'box' or 'line' or 'none'
toggle_selector.RS = RectangleSelector(current_ax, region_select_callback,
                                    drawtype='box', useblit=True,
                                    button=[1, 3],  # don't use middle button
                                    minspanx=5, minspany=5,
                                    spancoords='pixels',
                                    interactive=True)
plt.connect('key_press_event', toggle_selector)
plt.show()

In [ ]:
# If you do **not** want to crop the video, skip this cell.
%matplotlib inline
video_orig = video.copy()
video_gray_orig = video_gray.copy()
x1, x2, y1, y2 = coord
video = video_orig[:, y1:y2, x1:x2]
video_gray = video_gray_orig[:, y1:y2, x1:x2]
plt.imshow(video[0])
plt.title("Cropped Video")
plt.show()

### 1. Find valid region in the field-of-view (FoV)

In [ ]:
background = np.median(video_gray, axis=0)
valid_region, pix_height, pix_width = find_valid_region(background)
plt.figure(figsize=(12,5))
plt.subplot(121)
plt.imshow(background, cmap='gray')
plt.axis('off')
plt.title('Background')
plt.subplot(122)
plt.imshow(valid_region, cmap='gray')
plt.axis('off')
plt.title('Valid region')
plt.show()
perpix_dist = ((height / pix_height) + 
               (width / pix_width)) / 2.0

### 2. Process and save the video

In [ ]:
def process_frame(i):
    center = segment_image(video_gray[i]-background, valid_region, relative_brightness=True)
    center_frame = np.zeros(video_gray[i].shape, dtype=np.uint8)
    if len(center)==2:
        center_frame[center[0], center[1]] = 1
    return center, center_frame

p = multiprocessing.Pool(num_cores)
start = time.perf_counter()
results = p.map(process_frame, list(range(num_frames)))
all_centers = [x[0] for x in results if x[0] != []]
all_frames = [x[1] for x in results]
center_video = np.stack(all_frames, 0)
end = time.perf_counter()
print("Animal tracking time: %.2f s" % (end-start))

# save the composite video
save_video(video, video_gray, center_video, track=True, 
           frame_rate=frame_rate*kwargs["subsample"])

In [ ]:
%matplotlib inline
travel_length = 0
temp = np.ones(video_gray[0].shape, dtype=np.uint8)*255
temp[0,0] = 0
ditance_map = np.zeros(temp.shape, dtype=np.float64)
plt.imshow(temp, cmap='gray')
for j in range(len(all_centers)-1):
    dist = center2dist(all_centers[j], all_centers[j+1])
    if dist < 100:
        travel_length += dist
        ditance_map[all_centers[j][0], all_centers[j][1]] += dist
        plt.plot([all_centers[j][1], 
                  all_centers[j+1][1]], 
                 [all_centers[j][0],
                  all_centers[j+1][0]], 'r', alpha=0.3, linewidth=2)
plt.axis('on')
plt.title('Track of the animal')
plt.show()
print("Travel length is %.2f pixels, which is %.2f cm." % (travel_length, travel_length*perpix_dist))

### 3. Interactive measurement

In [ ]:
%matplotlib notebook
heatmap = center_video.max(0)
heatmap = dilation(heatmap, disk(3))
heatmap = np.stack([heatmap, 
                    np.zeros_like(heatmap),
                    np.zeros_like(heatmap)], 2)
heatmap = (heatmap*255).astype(np.uint8)
heatmap = np.clip(heatmap, a_min=0, a_max=255)
heatmap = np.maximum(heatmap, video[0])

# Interactive measurement
%matplotlib notebook
interactive_measure(heatmap, center_video, ditance_map, 
                    perpix_dist, frate=frame_rate)

In [ ]:
## This cell is intentionally left blank.